# Proyecto: Métodos de Interpolación
### Análisis y Predicción de Datos Meteorológicos

Interpolacion (BREVE EXPLICACION)

Los datos sumisnistrados son de la ciudad __Basilea__, en Suiza de 2023; y fueron proporcionados por la pagina __meteoblue__: https://www.meteoblue.com/es/tiempo/archive/export se debe tener en cuenta que los datos se encuentran en un archivo __CSV__, por lo se debe instalar los paquetes necesarios para poder abrir el documento apropiadamente en JULIA.

In [ ]:
#INSTALACION DE PAQUETES
#Instalacion de paquete para usar archivo CSV
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")

#Intalacion de plot, permite graficar
using Pkg
Pkg.add("Plots")

Para acceder al archivo se importa el paquete __CSV__ y __DataFrames__ empleando la palabra '__using__' como se muestra en el codigo.

In [ ]:
#EJEMPLO PARA ABRIR Y LEER UN ARCHIVO
# Importar paquetes
using CSV
using DataFrames

# Leer el archivo CSV
archivo_csv = "data_Basilea.csv" #nombre del archivo
datos = CSV.read(archivo_csv, DataFrame) #lectura del archivo

# Mostrar las primeras filas del DataFrame
println("Datos Generales:") #datos generales del archivo
println(first(datos, 8)) #0-8 filas


In [ ]:
#EJEMPLO A SEGUIR PARA GRAFICAS
# Importar el paquete Plots
using Plots

# Lista de datos (temperaturas)
temperaturas = [20, 21, 19, 22, 23, 20, 21]

# Crear un vector de días (suponiendo que cada valor en la lista corresponde a un día)
dias = 1:length(temperaturas)

# Graficar los datos
plot(dias, temperaturas, label="Temperatura", xlabel="Días", ylabel="Temperatura (°C)", title="Temperaturas Diarias")

### Interpolación de Taylor

Informacion general en base a la teroria
* Se estable un rango de puntos conocidos, que permiten aproximar valores.
* Se construye un polinomio Pn(x) de grado menor o igual que n. (Polinomio de Taylor)

Para mayor informacion revisar el siguiente link: https://rua.ua.es/dspace/bitstream/10045/102428/1/tema2.pdf

Aplicando teoria en julia: <br>
Mediante los datos de entrada que serian los dias de una semana, se puede busca predecir un dia especifico, en este caso se tomara la segunda semana de 2023 que va el del 8 al 15 de enero, y el dia que se desea predecir es el 16 de enero.

In [24]:
# Importar paquetes
using CSV
using DataFrames
using LinearAlgebra 
using Plots


# Leer el archivo CSV
archivo = "data_Basilea.csv" #nombre del archivo
datos = CSV.read(archivo, DataFrame) #lectura del archivo

#puntos seleccionados de una semana
#'parse.' convierte str a float
#'collect' convierte en vector-lista
vector = collect(parse.(Float64, datos[10:16, 2]))


#ACOMODAR/ AGREGAR RANDON PART FOR WEEKS/ ADD PLOT DATA OF INTERPOLACION

# Función para calcular la derivada de primer orden (diferencia finita)
function primera_derivada(x)
    derivada = diff(x)

    return derivada
end

# Calcular la serie de Taylor
function taylor_interpolacion(x, punto_a_predecir)
    # Orden de la serie de Taylor (para este ejemplo usamos 2)
    orden = 2
    
    # Calcular derivadas
    primera_deriv = primera_derivada(x)
    segunda_deriv = primera_derivada(primera_deriv)
    
    # Último valor conocido (base para la expansión de Taylor)
    ultimo_valor = x[end]
    
    # Predicción usando serie de Taylor
    prediccion = ultimo_valor
    if orden >= 1
        # Termino de la primera derivada
        prediccion += primera_deriv[end]
    end
    if orden >= 2
        # Termino de la segunda derivada
        prediccion += segunda_deriv[end] / 2
    end
    
    return prediccion
end

# Predicción del día 8
prediccion_dia_8 = taylor_interpolacion(vector, 8) #10 




#cantidad de dias 1:length(vector)

#grafica puntos seleccionados
#plot(dias, 1:length(vector), label="Temperatura", xlabel="Días", ylabel="Temperatura (°C)", title="Temperaturas Diarias de una semana")

9.525244999999998

### Interpolación de Lagrange

### Interpolación de Hermite

### Interpolación de Polinomicas a Trozos